# Preperation
```
$ # Make and move to epidemiology (only execute if it does not exists)
$ mkdir /home/labuser/epidemiology && cd "$_" 
$ 
$ # Download epi.csv and top hits from the papers
$ wget https://transfer.sh/Jl0tF/epi.csv.zip 
$ wget https://transfer.sh/DeUjv/papers_top_hits.zip  
$
$ # Get and execute the checksum
$ wget https://raw.githubusercontent.com/Infomdss2018/infomdss/master/tutorial_7/checksum/epi.csv.zip.sha1
$ 
$ wget https://raw.githubusercontent.com/Infomdss2018/infomdss/master/tutorial_7/checksum/papers_top_hits.zip.sha1 
$ 
$ sha1sum -c epi.csv.zip.sha1
$ sha1sum -c papers_top_hits.zip.sha1
$ 
$ # Unzip epi.csv data and give permissions
$ unzip epi.csv.zip -d data/
$ unzip papers_top_hits.zip -d data/
$
$ # give permission to the data folder
$ chmod 777 data/*
```

# Step 3: load the data-files into your notebook

In [30]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("epidemiology").getOrCreate()
sc = spark.sparkContext

# Note that only the SAMPLE data is used, if you want to test the real data you can uncommend the following lines
chd_paper_path = "/home/labuser/epidemiology/data/CHD_paper.csv"
men_paper_path = "/home/labuser/epidemiology/data/MENO_paper.csv"
t2d_paper_path = "/home/labuser/epidemiology/data/T2D_paper.csv"

epi_path = "/home/labuser/epidemiology/data/epi.csv"

*1 Provide the top hits for each of the outcomes (CAD, T2D and age at menopause)*

In [5]:
# use the data frames from previouse tutorial
chd_path = "/home/labuser/epidemiology/data/CHD_coronary_heart_disease_SAMPLE.csv"
men_path = "/home/labuser/epidemiology/data/MENO_Menopause_HapMap2_DayNG2015_18112015_SAMPLE.csv"
t2d_path = "/home/labuser/epidemiology/data/T2D_diabetes_type_two_SAMPLE.csv"

# Note that only the SAMPLE data is used, if you want to test the real data you can uncommend the following lines
#chd_path = "/home/labuser/epidemiology/data/CHD_coronary_heart_disease.csv"
#men_path = "/home/labuser/epidemiology/data/MENO_Menopause_HapMap2_DayNG2015_18112015.csv"
#t2d_path = "/home/labuser/epidemiology/data/T2D_diabetes_type_two.csv"

chd_df = spark.read.option("delimiter",";").csv(chd_path, inferSchema=True, header=True)
men_df = spark.read.option("delimiter",";").csv(men_path, inferSchema=True, header=True)
t2d_df = spark.read.option("delimiter",";").csv(t2d_path, inferSchema=True, header=True)

In [6]:
# show the data to determine the column representing the outcome (p-value) of each hit
chd_df.show()
men_df.show()
t2d_df.show()

+----------------+---+---------+-------------+----------------+------------------+-----------+-----+---------+---------+---------+----------+---------+
|      markername|chr|  bp_hg19|effect_allele|noneffect_allele|effect_allele_freq|median_info|model|     beta|   se_dgc|    p_dgc|het_pvalue|n_studies|
+----------------+---+---------+-------------+----------------+------------------+-----------+-----+---------+---------+---------+----------+---------+
|       rs7540212|  1|157259747|            G|               T|          0.675984|      993.0|FIXED| 0.010379|0.0101002| 0.304137|  0.852783|       48|
|      rs35174775|  6| 24847434|            G|               A|          0.685355|   0.981575|FIXED|-0.003668|0.0102143|0.7195163|   0.85337|       47|
|      rs17717184| 16| 77829273|            C|               T|          0.836133|      971.0|FIXED|-0.008041|0.0127475|0.5281767|  0.375501|       46|
|       rs6896654|  5| 78024784|            T|               A|          0.785148|    0.

In [7]:
# you can sort on p-value to show the top hits
chd_df_sorted = chd_df.orderBy("p_dgc").cache()
men_df_sorted = men_df.orderBy("P").cache()
t2d_df_sorted = t2d_df.orderBy("P-value").cache()

chd_df_sorted.show()
men_df_sorted.show()
t2d_df_sorted.show()

+----------------+---+---------+-------------+----------------+------------------+-----------+-----+---------+---------+---------+----------+---------+
|      markername|chr|  bp_hg19|effect_allele|noneffect_allele|effect_allele_freq|median_info|model|     beta|   se_dgc|    p_dgc|het_pvalue|n_studies|
+----------------+---+---------+-------------+----------------+------------------+-----------+-----+---------+---------+---------+----------+---------+
|     rs148274710| 20| 11369643|            T|               A|           0.96762|    0.80692|FIXED|-0.085761|0.0304505|0.0048565|  0.906542|       45|
|       rs4764102| 12| 14702794|            G|               T|           0.89008|    0.64242|FIXED| 0.059495|0.0226424|0.0085991|  0.471049|       43|
|      rs11044100| 12| 18577377|            T|               C|          0.801858|        1.0|FIXED| 0.029016| 0.011508|0.0116899|   0.24046|       48|
|     rs141209575| 15| 45118164|            C|               T|          0.110322|      

*2 Check the overlap with the top hits mentioned in the papers or online.*

In [8]:
epi_df = spark.read.csv(epi_path).toDF('Chr:Position', 'markername', 'chr', 'bp_hg19', 'effect_allele', 
                                       'noneffect_allele', 'effect_allele_freq', 'median_info', 'model', 'beta',
                                       'se_dgc', 'p_dgc', 'het_pvalue', 'n_studies', 'allele1', 'allele2', 'HapMap_eaf',
                                       'effect', 'stderr', 'p', 'Allele1', 'Allele2', 'Effect', 'StdErr', 'P-value',
                                       'TotalSampleSize')

epi_df.show()

+------------+----------+---+---------+-------------+----------------+------------------+-----------+-----+---------+---------+---------+----------+---------+-------+-------+----------+------+------+-----+-------+-------+-------+------+-------+---------------+
|Chr:Position|markername|chr|  bp_hg19|effect_allele|noneffect_allele|effect_allele_freq|median_info|model|     beta|   se_dgc|    p_dgc|het_pvalue|n_studies|allele1|allele2|HapMap_eaf|effect|stderr|    p|Allele1|Allele2| Effect|StdErr|P-value|TotalSampleSize|
+------------+----------+---+---------+-------------+----------------+------------------+-----------+-----+---------+---------+---------+----------+---------+-------+-------+----------+------+------+-----+-------+-------+-------+------+-------+---------------+
|10:100473135| rs1932734| 10|100473135|            G|               A|          0.814632|        1.0|FIXED| 0.007945|0.0116568|0.4955087|  0.288415|       48|      a|      g|      0.14| -0.05|  0.03|0.049|      A|    

In [9]:
# open top hits as mentioned in the papers
chd_paper_df = spark.read.option("delimiter","\t").csv(chd_paper_path, inferSchema=True, header=True)
men_paper_df = spark.read.option("delimiter","\t").csv(men_paper_path, inferSchema=True, header=True)
t2d_paper_df = spark.read.option("delimiter","\t").csv(t2d_paper_path, inferSchema=True, header=True)

# join the data sets from the papers
t2d_men_paper_df = t2d_paper_df.join(men_paper_df, 'markername', 'inner').cache()
t2d_chd_paper_df = t2d_paper_df.join(chd_paper_df, 'markername', 'inner').cache()
men_chd_paper_df = men_paper_df.join(chd_paper_df, 'markername', 'inner').cache()

# join the data sets from the papers with the epi dataset
epi_men_paper_df = epi_df.join(men_paper_df, 'markername', 'inner').cache()
epi_t2d_paper_df = epi_df.join(t2d_paper_df, 'markername', 'inner').cache()
epi_chd_paper_df = epi_df.join(chd_paper_df, 'markername', 'inner').cache()

print ("Diabetes shares {0} hits with menopause.".format(t2d_men_paper_df.count()))
print ("Diabetes shares {0} hits with cardio.".format(t2d_chd_paper_df.count()))
print ("Menopause shares {0} hits with cardio.".format(men_chd_paper_df.count()))
print ("-------------")
print ("Epi shares {0} hits with menopause.".format(epi_men_paper_df.count()))
print ("Epi shares {0} hits with diabetes.".format(epi_t2d_paper_df.count()))
print ("Epi shares {0} hits with cardio.".format(epi_chd_paper_df.count()))

Diabetes shares 0 hits with menopause.
Diabetes shares 0 hits with cardio.
Menopause shares 0 hits with cardio.
-------------
Epi shares 54 hits with menopause.
Epi shares 4 hits with diabetes.
Epi shares 4 hits with cardio.


*3 Is there overlap between the top hits (for instance the 50 hits with the lowest p-values per trait)? How large is this overlap? Which SNPs are those and what is the ranking p-value for each of the traits for those SNPs (you can think of a table with SNPid, ranking per trait and pvalue per trait, so 7 columns).*

For this question, there are three tables generated, one for every threat. The ranking is based on the p-values of the different SNP's and starts at 0, which is associated with the lowest p-value. So a large number in ranking means a distant relation.

In [19]:
# add ranks to sorted data
def map_ranking(line):
    return [int(line[1])] + list(line[0])

# create headers for the rank tables
chd_df_header = chd_df_sorted.schema.names
men_df_header = men_df_sorted.schema.names
t2d_df_header = t2d_df_sorted.schema.names

# create the rank tables (trait plus rank)
chd_rank_df = chd_df_sorted.rdd.zipWithIndex().map(map_ranking).toDF(["Rank"] + chd_df_header).cache()
men_rank_df = men_df_sorted.rdd.zipWithIndex().map(map_ranking).toDF(["Rank"] + men_df_header).cache()
t2d_rank_df = t2d_df_sorted.rdd.zipWithIndex().map(map_ranking).toDF(["Rank"] + t2d_df_header).cache()

# select top 50 hits 
chd_rank_50_df = chd_rank_df.limit(50)
men_rank_50_df = men_rank_df.limit(50)
t2d_rank_50_df = t2d_rank_df.limit(50)

# find overlap with epi-dataset
epi_chd_rank_50_df = epi_df.join(chd_rank_50_df, 'markername', 'inner').cache()
epi_men_rank_50_df = epi_df.join(men_rank_50_df, 'markername', 'inner').cache()
epi_t2d_rank_50_df = epi_df.join(t2d_rank_50_df, 'Chr:Position', 'inner').cache()

# count overlapping items
print ("Cardio rank 50 has {0} overlapping hits with epi.".format(epi_chd_rank_50_df.count()))
print ("Diabetes rank 50 has {0} overlapping hits with epi.".format(epi_men_rank_50_df.count()))
print ("Menopause rank 50 has {0} overlapping hits with epi.".format(epi_t2d_rank_50_df.count()))

# create dataframe with overlapping items
def create_ranking_table(epi_lit_df):
    data_headers = "SNPid", "cad_p","cad_rank","meno_p","meno_rank","T2D_p","T2D_rank"
    data_matrix = []

    for n in epi_lit_df.rdd.collect():
        marker = n["markername"]
        chr_ps = n["Chr:Position"]
        query_cata = chd_rank_df.filter(chd_rank_df["markername"] == marker).select("p_dgc", "rank").first()
        query_meno = men_rank_df.filter(men_rank_df["markername"] == marker).select("p", "rank").first()
        query_meta = t2d_rank_df.filter(t2d_rank_df["Chr:Position"] == chr_ps).select("P-value", "rank").first()
        
        try:
            data_matrix.append((marker, query_cata[0], query_cata[1], query_meno[0], query_meno[1], 
                                query_meta[0], query_meta[1]))
        except TypeError:
            continue
    
    return sqlContext.createDataFrame(data_matrix, data_headers)

epi_chd_rank_50_table = create_ranking_table(epi_chd_rank_50_df).orderBy("cad_p").cache()
epi_men_rank_50_table = create_ranking_table(epi_men_rank_50_df).orderBy("meno_p").cache()
epi_t2d_rank_50_table = create_ranking_table(epi_t2d_rank_50_df).orderBy("T2D_rank").cache()

# display top 50 overlap
print ("=== {0} ===".format("Cardio"))
epi_chd_rank_50_table.show()

print ("=== {0} ===".format("Menopause"))
epi_men_rank_50_table.show()

print ("=== {0} ===".format("Diabetes"))
epi_t2d_rank_50_table.show()

Cardio rank 50 has 17 overlapping hits with epi.
Diabetes rank 50 has 49 overlapping hits with epi.
Menopause rank 50 has 16 overlapping hits with epi.
=== Cardio ===
+----------+---------+--------+------+---------+-----+--------+
|     SNPid|    cad_p|cad_rank|meno_p|meno_rank|T2D_p|T2D_rank|
+----------+---------+--------+------+---------+-----+--------+
| rs6835662|0.0158637|       5|  0.47|       47| 0.89|      85|
|rs11858005|0.0544269|       9|  0.74|       71| 0.38|      33|
|rs10110110| 0.137665|      16|  0.75|       72| 0.62|      66|
| rs4280510|0.1618271|      20|   0.9|       90|  0.9|      86|
| rs2898190|0.1850134|      23|  43.0|      103| 0.88|      82|
| rs7648152|0.3153011|      40|  0.72|       69|  0.7|      70|
| rs3814241|0.3834816|      43|  0.84|       86| 0.22|      16|
|rs13406567|0.3852794|      44|  0.62|       63| 0.36|      31|
+----------+---------+--------+------+---------+-----+--------+

=== Menopause ===
+---------+---------+--------+------+---------

Displayed below are three different data frames, each data frame contains the p-value and ranking for an overlapping SNPid from one of the papers. For example in the CAD paper are four SNPid's that are also present in the meno and T2D dataset.

For every data-set the entire p-value ranking is described so we can now compare, say rs663129, which takes a relatively high ranking with the CAD-dataset (as expected) but also with the T2D-dataset (interesting) and fairly less with the MENO-dataset. So rs663129 might be closely related to CAD and T2D but not with MENO.

Note, when using the EXAMPLE data-sets, all tables will be empty.

In [29]:
from pyspark.sql.types import *

# Create default dataframe schema
data_schema = StructType([StructField("SNPid", StringType(), True), StructField("p_lit", DoubleType(), True),
                             StructField("cad_p", DoubleType(), True), StructField("cad_rank", LongType(), True), 
                             StructField("meno_p", DoubleType(), True), StructField("meno_rank", LongType(), True), 
                             StructField("T2D_p", DoubleType(), True), StructField("T2D_rank", LongType(), True)])

def create_ranking_table(epi_lit_df):
    data_headers = "SNPid", "p_lit","cad_p","cad_rank","meno_p","meno_rank","T2D_p","T2D_rank"
    data_matrix = []

    for n in epi_lit_df.rdd.collect():
        marker = n["markername"]
        chr_ps = n["Chr:Position"]
        litr_p = n["p-value"] # p-value from paper
        query_cata = chd_rank_df.filter(chd_rank_df["markername"] == marker).select("p_dgc", "rank").first()
        query_meno = men_rank_df.filter(men_rank_df["markername"] == marker).select("p", "rank").first()
        query_meta = t2d_rank_df.filter(t2d_rank_df["Chr:Position"] == chr_ps).select("P-value", "rank").first()
        
        try:
            data_matrix.append((marker, litr_p, query_cata[0], query_cata[1], query_meno[0], query_meno[1], 
                                query_meta[0], query_meta[1]))
        except TypeError:
            continue
    
    return sqlContext.createDataFrame(data_matrix, data_schema)

men_ranking_table = create_ranking_table(epi_men_paper_df).orderBy("p_lit").cache()
t2d_ranking_table = create_ranking_table(epi_t2d_paper_df).orderBy("p_lit").cache()
chd_ranking_table = create_ranking_table(epi_chd_paper_df).orderBy("p_lit").cache()

print ("=== {0} ===".format("Cardio"))
chd_ranking_table.show()

print ("=== {0} ===".format("Diabetes"))
t2d_ranking_table.show()

print ("=== {0} ===".format("Menopause"))
men_ranking_table.show()

=== Cardio ===
+-----+-----+-----+--------+------+---------+-----+--------+
|SNPid|p_lit|cad_p|cad_rank|meno_p|meno_rank|T2D_p|T2D_rank|
+-----+-----+-----+--------+------+---------+-----+--------+
+-----+-----+-----+--------+------+---------+-----+--------+

=== Diabetes ===
+-----+-----+-----+--------+------+---------+-----+--------+
|SNPid|p_lit|cad_p|cad_rank|meno_p|meno_rank|T2D_p|T2D_rank|
+-----+-----+-----+--------+------+---------+-----+--------+
+-----+-----+-----+--------+------+---------+-----+--------+

=== Menopause ===
+-----+-----+-----+--------+------+---------+-----+--------+
|SNPid|p_lit|cad_p|cad_rank|meno_p|meno_rank|T2D_p|T2D_rank|
+-----+-----+-----+--------+------+---------+-----+--------+
+-----+-----+-----+--------+------+---------+-----+--------+

